In [1]:
import sys, os
import numpy as np
import pandas as pd
from utils.utils import read_txt, read_txt_np_int

# INPUT

In [3]:
inputfilename = './inputs/day10A.txt'

inputdata = read_txt(inputfilename)

testdata = ["89010123",
"78121874",
"87430965",
"96549874",
"45678903",
"32019012",
"01329801",
"10456732"]

In [59]:
data_to_use = inputdata

elevation_map = np.array([[char for char in line ] for line in data_to_use]).astype(int)
elevation_map


array([[8, 9, 8, ..., 6, 7, 6],
       [7, 8, 7, ..., 5, 8, 9],
       [0, 1, 6, ..., 4, 3, 9],
       ...,
       [3, 6, 9, ..., 6, 6, 8],
       [4, 5, 2, ..., 7, 8, 9],
       [5, 4, 3, ..., 2, 1, 0]], shape=(40, 40))

## PART 1

In [46]:
def find_start_points(elevation_map, elevation=0):
    return [(np.where(elevation_map == elevation)[0][i], np.where(elevation_map == elevation)[1][i]) for i in range(len(np.where(elevation_map == elevation)[0]))]

In [43]:
find_start_points(elevation_map)

[(np.int64(0), np.int64(2)),
 (np.int64(0), np.int64(4)),
 (np.int64(2), np.int64(4)),
 (np.int64(4), np.int64(6)),
 (np.int64(5), np.int64(2)),
 (np.int64(5), np.int64(5)),
 (np.int64(6), np.int64(0)),
 (np.int64(6), np.int64(6)),
 (np.int64(7), np.int64(1))]

In [36]:
hor_ver_deviations = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]])

def find_next_step(elevation_map, position):
    next_steps = set()
    for deviation in hor_ver_deviations:
        next_position = (position[0] + deviation[0], position[1] + deviation[1])
        if next_position[0] < 0 or next_position[1] < 0:
            continue
        try:
            if elevation_map[next_position] == elevation_map[position] + 1:
                next_steps.add(next_position)
        except IndexError:
            pass
    return next_steps

def find_next_step_list(elevation_map, positions):
    next_steps = set()
    for position in positions:
        next_steps = next_steps.union(find_next_step(elevation_map, position))
    return next_steps

def reach_summit(elevation_map, starting_position, final_height = 9):
    current_elevation = elevation_map[starting_position]
    current_positions = set([starting_position])
    # Increase one elevation level at a time and check we have reached that level
    while current_elevation < final_height and len(current_positions) > 0:
        current_positions = find_next_step_list(elevation_map, current_positions)
        #print(f"Found {current_positions} positions at elevation {current_elevation}")
        current_elevation += 1
    for final_position in current_positions:
        assert(elevation_map[final_position] == final_height)
    return current_positions


In [37]:
# Solve part 1
trailheads = 0
for starting_position in find_start_points(elevation_map):
    current_trailheads = len(reach_summit(elevation_map, starting_position))
    #print(f"Found {current_trailheads} trailheads at {starting_position}")
    trailheads += current_trailheads
print(trailheads)

482


In [ ]:
reach_summit(elevation_map, (6, 0))

Found {(np.int64(6), np.int64(1)), (np.int64(7), np.int64(0)), (np.int64(6), np.int64(-1))} positions at elevation 0
Found {(np.int64(7), np.int64(-1)), (np.int64(5), np.int64(-1)), (np.int64(5), np.int64(1))} positions at elevation 1
Found {(np.int64(7), np.int64(-2)), (np.int64(5), np.int64(0)), (np.int64(4), np.int64(-1))} positions at elevation 2
Found {(np.int64(4), np.int64(0)), (np.int64(3), np.int64(-1))} positions at elevation 3
Found {(np.int64(2), np.int64(-1)), (np.int64(4), np.int64(1))} positions at elevation 4
Found {(np.int64(3), np.int64(1)), (np.int64(2), np.int64(-2)), (np.int64(4), np.int64(2))} positions at elevation 5
Found {(np.int64(1), np.int64(-2)), (np.int64(3), np.int64(-2)), (np.int64(2), np.int64(1)), (np.int64(4), np.int64(3))} positions at elevation 6
Found {(np.int64(4), np.int64(4)), (np.int64(1), np.int64(-3)), (np.int64(1), np.int64(1)), (np.int64(3), np.int64(-3)), (np.int64(2), np.int64(0))} positions at elevation 7
Found {(np.int64(0), np.int64(1)

{(np.int64(0), np.int64(1)),
 (np.int64(2), np.int64(-3)),
 (np.int64(3), np.int64(-4)),
 (np.int64(3), np.int64(0)),
 (np.int64(3), np.int64(4)),
 (np.int64(4), np.int64(-3)),
 (np.int64(4), np.int64(5)),
 (np.int64(5), np.int64(4))}

## PART 2

In [60]:
def find_previous_step_rating(elevation_map, rating_map, position):
    next_positions = set()
    current_elevation = elevation_map[position]
    current_trail_rating = rating_map[position]
    assert(current_elevation > 0)
    for deviation in hor_ver_deviations:
        next_position = (position[0] + deviation[0], position[1] + deviation[1])
        if next_position[0] < 0 or next_position[1] < 0:
            continue
        try:
            # Increase the trail rating for all previous positions
            if elevation_map[next_position] == current_elevation - 1:
                rating_map[next_position] += current_trail_rating
                next_positions.add(next_position)
        except IndexError:
            pass
    return rating_map, next_positions

def find_previous_step_rating_list(elevation_map, rating_map, positions):
    next_positions = set()
    for position in positions:
        rating_map, next_positions_iter = find_previous_step_rating(elevation_map, rating_map, position)
        next_positions = next_positions.union(next_positions_iter)
    return rating_map, next_positions

def find_summit_rating(elevation_map, final_height = 9):
    rating_map = (elevation_map == final_height).astype(int)
    next_positions = find_start_points(elevation_map, final_height)
    current_elevation = final_height
    while current_elevation > 0:
        rating_map, next_positions = find_previous_step_rating_list(elevation_map, rating_map, next_positions)
        current_elevation -= 1
    rating = np.sum(rating_map * (elevation_map == 0).astype(int))
    return rating, rating_map, next_positions, current_elevation

    

In [56]:
(elevation_map == 9).astype(int)

array([[0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [61]:
# Solve part 2
find_summit_rating(elevation_map)

(np.int64(1094),
 array([[1, 1, 1, ..., 1, 1, 1],
        [2, 1, 3, ..., 1, 1, 1],
        [6, 3, 3, ..., 1, 1, 1],
        ...,
        [2, 2, 1, ..., 1, 1, 1],
        [2, 2, 2, ..., 1, 1, 1],
        [0, 2, 2, ..., 1, 1, 1]], shape=(40, 40)),
 {(np.int64(0), np.int64(3)),
  (np.int64(0), np.int64(7)),
  (np.int64(0), np.int64(17)),
  (np.int64(0), np.int64(30)),
  (np.int64(0), np.int64(32)),
  (np.int64(1), np.int64(4)),
  (np.int64(1), np.int64(12)),
  (np.int64(1), np.int64(16)),
  (np.int64(1), np.int64(24)),
  (np.int64(1), np.int64(25)),
  (np.int64(1), np.int64(33)),
  (np.int64(2), np.int64(0)),
  (np.int64(2), np.int64(4)),
  (np.int64(2), np.int64(9)),
  (np.int64(2), np.int64(17)),
  (np.int64(2), np.int64(20)),
  (np.int64(3), np.int64(5)),
  (np.int64(4), np.int64(0)),
  (np.int64(4), np.int64(17)),
  (np.int64(4), np.int64(20)),
  (np.int64(4), np.int64(29)),
  (np.int64(4), np.int64(37)),
  (np.int64(5), np.int64(2)),
  (np.int64(5), np.int64(24)),
  (np.int64(5), np.